[link text](https://)
# 📌 Scraping de Facebook

Este notebook realiza scraping en **Facebook** para recorrer distintas **ciudades** y varias **palabras clave**.

**Incluye:**
- Uso de Playwright para la automatización de navegación en Facebook, incluyendo login con sesión guardada y aceptación de cookies.
- Implementación de scroll infinito para cargar más resultados en las páginas y recolectar publicaciones.
- Extracción de texto y fecha de cada publicación, junto con la asociación a una ciudad y una keyword.
- Ejecución en forma asíncrona (async/await) para procesar múltiples combinaciones de ciudad × keyword de manera más rápida y eficiente.


# Scraping de Facebook (Playwright)

## 🧰 Librerías e importaciones

In [ ]:
import asyncio
from playwright.async_api import async_playwright
import json
from datetime import datetime

## 🔐 Guardar sesión en Facebook (Playwright)

In [ ]:
async def guardar_sesion_facebook():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        await page.goto("https://www.facebook.com/login")
        print("Inicia sesión manualmente...")

        try:
            # Espera a que aparezca el botón "Inicio"
            await page.wait_for_selector('a[role="link"][aria-label="Inicio"]', timeout=120_000)
            print("Botón 'Inicio' detectado. Navegando al feed principal...")

            # Haz clic en "Inicio"
            await page.click('a[role="link"][aria-label="Inicio"]')

            # Guardar sesión
            await context.storage_state(path="facebook_sesion.json")
            print("Sesión guardada correctamente en 'facebook_sesion.json'.")

        except Exception as e:
            print(f"Error durante la navegación tras login: {e}")

        print("Puedes cerrar el navegador manualmente cuando termines.")

# Ejecuta esto una vez:
await guardar_sesion_facebook()

Inicia sesión manualmente...
Error durante la navegación tras login: Page.wait_for_selector: Target page, context or browser has been closed
Call log:
  - waiting for locator("a[role=\"link\"][aria-label=\"Inicio\"]") to be visible

Puedes cerrar el navegador manualmente cuando termines.


## 🕷️ Scraper: buscar publicaciones por consulta

In [ ]:
async def scrape_facebook_posts(query, max_posts=50):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context(storage_state="facebook_sesion.json")
        page = await context.new_page()

        # Armar URL de búsqueda en Facebook
        url = f"https://www.facebook.com/search/posts/?q={query.replace(' ', '%20')}"
        await page.goto(url)
        print(f"Buscando: {query}")
        await asyncio.sleep(5)

        posts = set()
        scrolls = 0

        while len(posts) < max_posts and scrolls < 30:
            await page.mouse.wheel(0, 5000)
            await asyncio.sleep(3)

            post_elements = await page.query_selector_all('[data-ad-preview="message"]')
            for el in post_elements:
                try:
                    text = await el.inner_text()
                    posts.add(text.strip())
                except:
                    continue

            scrolls += 1

        await browser.close()
        return list(posts)[:max_posts]

## ⚙️ Parámetros de búsqueda (ciudades y palabras clave)

In [ ]:
ciudades = ['Tenerife', 'Barcelona', 'Madrid', 'Malaga', 'Gran Canaria', 'Seville', 'Valencia', 'Palma de Mallorca']
palabras_clave = ["turismo", "viajar", "qué hacer", "actividades", "atracciones",
    "hoteles", "restaurantes", "experiencias", "vacaciones", "monumentos", "lugares"]

## 🔁 Orquestación: combinaciones ciudad × palabra clave

In [ ]:
async def recolectar_facebook_posts_por_combinacion():
    for ciudad in ciudades:
        print(f"Procesando ciudad: {ciudad}")
        posts_totales = []

        for keyword in palabras_clave:
            await asyncio.sleep(3)
            query = f"{ciudad} {keyword}"
            print(f"Buscando: {query}")

            try:
                posts = await scrape_facebook_posts(query, max_posts=200)
            except Exception as e:
                print(f"Error con '{query}': {e}")
                posts = []

            for texto in posts:
                posts_totales.append({
                    "ciudad": ciudad,
                    "keyword": keyword,
                    "query": query,
                    "texto": texto
                })

            print(f"{len(posts)} posts extraídos de '{query}'")

        # Guardar en archivo por ciudad
        timestamp = datetime.now().strftime("%Y%m%d")
        filename = f"{ciudad.replace(' ', '_')}_facebook_posts_{timestamp}.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(posts_totales, f, ensure_ascii=False, indent=2)

        print(f"Guardado {len(posts_totales)} posts en {filename}\n")

## ▶️ Ejecución

In [ ]:
await recolectar_facebook_posts_por_combinacion()

Procesando ciudad: Tenerife
Buscando: Tenerife turismo
Error con 'Tenerife turismo': [Errno 2] No such file or directory: 'facebook_sesion.json'
0 posts extraídos de 'Tenerife turismo'
Buscando: Tenerife viajar
Error con 'Tenerife viajar': [Errno 2] No such file or directory: 'facebook_sesion.json'
0 posts extraídos de 'Tenerife viajar'
Buscando: Tenerife qué hacer
Error con 'Tenerife qué hacer': [Errno 2] No such file or directory: 'facebook_sesion.json'
0 posts extraídos de 'Tenerife qué hacer'
Buscando: Tenerife actividades
Error con 'Tenerife actividades': [Errno 2] No such file or directory: 'facebook_sesion.json'
0 posts extraídos de 'Tenerife actividades'
Buscando: Tenerife atracciones
Error con 'Tenerife atracciones': [Errno 2] No such file or directory: 'facebook_sesion.json'
0 posts extraídos de 'Tenerife atracciones'
Buscando: Tenerife hoteles
Error con 'Tenerife hoteles': [Errno 2] No such file or directory: 'facebook_sesion.json'
0 posts extraídos de 'Tenerife hoteles'
Bus